In [32]:
# import libraries for analysis
import numpy as np
import json
import pandas as pandas
import matplotlib.pyplot as pyplot
from pandas.io.json import json_normalize

import re
from textblob import TextBlob

In [7]:
dbfile = './data/db2.json'
json_data = json.load(open(dbfile))
data = json_normalize(json_data)
print(len(data))

data = data.iloc[::2] #remove every other entry (because of data collection issues)
print(len(data))
# data.drop_duplicates(subset='doc._rev', inplace=True) #remove exact duplicates
data.reset_index(drop=True, inplace=True) #reindex the dataframe
data.drop(labels=['doc._id', 'doc._rev', 'id', 'key', 'value.rev', 'doc.attachment_download ', 'doc.attachment_download -href', 'doc.attachment_name'], inplace=True, axis='columns')

33064
16532


In [25]:
data.loc[:]

,doc.category,doc.city,doc.comment_body,doc.country,doc.name,doc.state,doc.zip
0,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Parent/Relative,Heather Hirsch,MN,55016
1,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Other,Maryann Decker,UT,84737
2,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Other,Greg Lofgren,WI,53704
3,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Other,Barbara Thomborson,MN,55408
4,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Other,Monique Beriau,None,NaN
5,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Other,Gerald Kline,NY,10065
6,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Parent/Relative,Melvin Hughes,TN,38583
7,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Individual,Lea Morgan,MA,01201
8,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Individual,Phyllis Chavez,CA,90405
9,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Parent/Relative,Niki Hoyle,MI,48642


In [8]:
# remove links and special characters
def remove_links(comment):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(comment)).split())

In [40]:
data['doc.comment_body'] = data['doc.comment_body'].map(remove_links)

In [44]:
data['sentiment'] = data.apply(lambda x: TextBlob(str(x['doc.comment_body'])).sentiment.polarity, axis=1)
with pandas.option_context('display.max_columns', 14, 'display.max_colwidth', 5000):
    display(data.sample(100))

doc.category  \
12869  Four-Year Public Institution of Higher Education   
9322                                               None   
6960                                               None   
3100                                               None   
637                                                None   
13840                                        Individual   
8484                                    Parent/Relative   
13923                                        Individual   
14727                                              None   
3840                                               None   
5142                                               None   
10335                                              None   
4023                                               None   
8681                                               None   
12649                                        Individual   
8650                     National Advocacy Organization   
3533                                               None   
13091                                              None   
5698                                               None   
6112                                               None   
3992                                               None   
2330                                               None   
12010                                              None   
6643                                               None   
5344                                               None   
15822                                              None   
3088                                               None   
1252                                               None   
11458                                        Individual   
10700                    National Advocacy Organization   
...                                                 ...   
15201                                              None   
6082                                               None   
8649                     National Advocacy Organization   
11867                                           Federal   
1346                                               None   
534                                                None   
15219                                              None   
8035                                               None   
10189                                              None   
4433                                               None   
13525                                              None   
14232                                              None   
1145                                               None   
2196                                               None   
12443                   Institution of Higher Education   
14986                                              None   
16491                                              None   
9899                                    Parent/Relative   
15396                                              None   
4144                                               None   
8938                                               None   
15314                                           Teacher   
5265                                               None   
14628                                              None   
3893                                               None   
5834                                               None   
244                                                None   
4630                                               None   
15304                        BLACK SKEPTICS LOS ANGELES   
8572                                    Parent/Relative   

                                          doc.city  \
12869                                 Fort Collins   
9322                                    Individual   
6960                                 United States   
3100                                 United States   
637                                  United States   
13840                                     Narberth   
8484  

The below number is the average sentiment for all comments and the comment that is closest in sentiment to the average

In [72]:
avg_sentiment = data['sentiment'].sum() / len(data.index)
print(avg_sentiment)

closest_sentiment = (np.abs(data['sentiment'] - avg_sentiment)).idxmin()
print(closest_sentiment)
with pandas.option_context('display.max_colwidth', 5000):
    display(data.iloc[[closest_sentiment]])

0.126676739069
11354


,doc.category,doc.city,doc.comment_body,doc.country,doc.name,doc.state,doc.zip,sentiment
11354,Individual,Dover,I raised two daughters who are dyslexic but all through school we had no idea what the problem was and I am a public school educator It wasn t until my girls were grown and I had volunteered as a tutor for illiterate adults that I discovered dyslexia and then I realized that is what my girls had been dealing with all through school The most unfortunate part is that the younger daughter is adopted She came to us as a foster child and we ended up adopting her State money deferred at her adoption was available to us for assistance as needed for things such as learning disabilities etc If we had only known what she was dealing with we could have hired specialized tutors or purchased an Orton Gillingham reading and spelling system for home tutoring which likely could have helped her stay on track had we started early enough I know schools do not like to label kids but when it comes to dyslexia it is very important that people know what their children are dealing with in order to get the appropriate interventions going,United States,Velvet Siegel,DE,NaN,0.126667


Res
https://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/
https://web.stanford.edu/~jurafsky/slp3/6.pdf
